In [1]:
import pandas as pd 
import numpy as np 
import os
import urllib
import importlib

In [3]:
import rmsd_functions as rmsd 
importlib.reload(rmsd)

<module 'rmsd_functions' from '/data/jlu/OR_learning/rmsd_functions.py'>

In [6]:
AF_files = os.listdir('./AF_pdb/')

In [ ]:
# extract Uniprot ID to filter for ORs 
AF_id = []
for file in AF_files:
    if "pdb" in file:
        AF_id.append(file.split('-')[1])
# pd.DataFrame(AF_id).to_csv('./AF_id.csv')

Uniprot id csv is copied and pasted to https://www.uniprot.org/mapping/
The converted Tab format is read below 

In [12]:
AF_table = pd.read_table('./AF_id_converted.tab')

In [15]:
# Extract for the ID that are Olfr gene names 
AF_table = AF_table[AF_table['Gene names'].str.contains("Olfr", na=False)]

In [16]:
# Remove none Olfr pdb files 
for file in AF_files:
#     statement checks in the current filename contains Olfr uniprot ID
    if [x for x in AF_table['Entry'] if x in file ] == []:
        os.remove("AF_pdb/"+file)

In [ ]:
# rename left over pdb files with gene names instead 
for file in os.listdir('./AF_pdb'):
    gene_name = AF_table[AF_table['Entry'] == file.split('-')[1]]['Gene names'].values[0]
    gene_name = gene_name.replace(" ","_")
    path = './AF_pdb/'+file
    os.rename(path, './AF_pdb/'+gene_name+'.pdb')

Since not all of the pdb files downloaded from AF has the same format. Likely due to when it's created. Read through the .pdb files, and only keep the rows that starts with "ATOM ..." 

In [3]:
pdb_files = os.listdir('./AF_pdb/Original/')

In [8]:
# For testing purpose. Olfr10 readable by read.pdb, wheras Olfr101 format is unintepreable by read.pdb in bio3d 
# pdb_files = ['Olfr101.pdb', 'Olfr10.pdb']

In [7]:
# opens pdb files, write a new files with only ATOM... 
for pdb in pdb_files: 
    atom_lines = []
    with open('./AF_pdb/Original/' + pdb) as f:
        lines = f.readlines()
        for line in lines:
            if line.split(" ")[0] == "ATOM":
                atom_lines.append(line)
    with open('./AF_pdb/' + pdb, 'w') as f:
        for line in atom_lines:
            f.write(line)


filter the pdb files so it only contains columns of 

ATOM resid atom_id aa x y z atom_id_simplified

In [44]:
with open('./AF_pdb/Olfr101.pdb') as f:
    Olfr101 = f.readlines()
with open('./AF_pdb/Olfr10.pdb') as f:
    Olfr10 = f.readlines()

In [8]:
# Rewrites the old-dated formate of pdb output from AF with 25 column in ATOM to the accepted readbale pdb format 
for pdb in pdb_files: 
    new_lines = []
    with open('./AF_pdb/' + pdb) as f: 
        lines = f.readlines()
        if len(lines[0].split()) == 25:
            for line in lines: 
                line = line.split()
                new_lines.append("{}  {}  {}  {}  {}  {}  {}  {}  {}  {}  {}  {}".format(line[0], line[1], line[3], 
                                                                     line[5], line[6], line[16], 
                                                                     line[10], line[11], line[12], 
                                                                     line[13], line[14], line[2]))
            with open('./AF_pdb/' + pdb, 'w') as f:
                for line in new_lines:
                    f.write(line + " \n")
                f.close()
